# Interacting with Mongo Atlas
- Notebook designed to specifically work with data related to collection `person`
---
---

# Imports

#### Standard library imports

In [1]:
import sys
sys.path.append("../")
import os
import json

#### Third party imports

In [2]:
import pandas as pd

#### Local application imports

In [3]:
%load_ext autoreload
%autoreload 2

from pkg_dir.config import *
from pkg_dir.src.utils import *
from pkg_dir.src.functions import *
from pkg_dir.src.parameters import *

### 

# Getting data

#### Preparing data from local source

In [4]:
## Online purchases data
path = '../pkg_dir/data/person.csv'
df_pers = pd.read_csv(path)
json_pers = df_pers.to_dict('records')

In [5]:
## Converting all the `Subclass` fields to json
for js in json_pers:
    js['Subclass'] = json.loads(js['Subclass'])

### 

# Database setup

In [6]:
## Cluster's client
mdb_client = create_and_test_mongodb_conn()

Pinged your deployment. You successfully connected to MongoDB!


In [7]:
## Creating/selecting database
mdb = mdb_client['beans_boutique_db']

In [8]:
## Creating/selecting collections
mdb_coll_pers = mdb['person']
mdb_coll_prod = mdb['product']

### 

# Entries upload

In [12]:
mdb_coll_pers.insert_many(json_pers)

### 

# Data queries

##### Query 1: testing query on nested field
- Finding consignor with ID equal to 9

In [20]:
q1 = {
  'Subclass.Consignor.Consignor_ID': 9
}

In [21]:
mdb_coll_pers.find(q1)[0]

{'_id': ObjectId('64506188a071c96ae9396d27'),
 'PID': 5,
 'FName': 'Jack',
 'MI': 'T',
 'LName': 'Mendez',
 'Address': 'Milvia 433 Berkeley CA',
 'Email_Addr': 'jt@gmail.com',
 'Phone': '673-374-3453',
 'Subclass': {'Employee': {},
  'Consignor': {'Consignor_ID': 9, 'Bank_Acct_No': 3822},
  'Customer': {},
  'Designer': {}}}

##### Query 2: test query on nested list
- Find employers that work on Wednesdays

In [13]:
q2 = {
  'Subclass.Employee.Working_Days': 'W'
}

In [26]:
for q_result in mdb_coll_pers.find(q2):
    display(q_result)

{'_id': ObjectId('64506188a071c96ae9396d23'),
 'PID': 1,
 'FName': 'John',
 'MI': 'R',
 'LName': 'Graft',
 'Address': 'Milvia 234 Berkeley CA',
 'Email_Addr': 'jrg@me.com',
 'Phone': nan,
 'Subclass': {'Employee': {'Employee_ID': '1',
   'SSN': '123',
   'Store': '1',
   'Union': '1',
   'Working_Days': ['M', 'W'],
   'Appraiser': '1',
   'Store_staff': '1'},
  'Consignor': {},
  'Customer': {'Customer_ID': '1', 'Membership': '543'},
  'Designer': {}}}

{'_id': ObjectId('64506188a071c96ae9396d24'),
 'PID': 2,
 'FName': 'Raul',
 'MI': 'G',
 'LName': 'Hernandez',
 'Address': 'Shattuck 444 Berkeley CA',
 'Email_Addr': 'rgh@gmail.com',
 'Phone': '673-374-3722',
 'Subclass': {'Employee': {'Employee_ID': 2,
   'SSN': 124,
   'Store': 2,
   'Union': 2,
   'Working_Days': ['M', 'W'],
   'Appraiser': 2,
   'Store_staff': 2},
  'Consignor': {},
  'Customer': {},
  'Designer': {}}}

##### Query 3: test query to find non empty fields
- Finding all the designers in the database

In [30]:
q3 = {
  'Subclass.Designer': {
      '$gt': {}
  }
}

In [31]:
for q_result in mdb_coll_pers.find(q3):
    display(q_result)

{'_id': ObjectId('64506188a071c96ae9396d26'),
 'PID': 4,
 'FName': 'Albert',
 'MI': 'E',
 'LName': 'Spivak',
 'Address': nan,
 'Email_Addr': nan,
 'Phone': nan,
 'Subclass': {'Employee': {},
  'Consignor': {},
  'Customer': {},
  'Designer': {'Designer_ID': 4}}}

{'_id': ObjectId('64506188a071c96ae9396d28'),
 'PID': 6,
 'FName': 'John',
 'MI': 'Z',
 'LName': 'Holz',
 'Address': 'Milvia 452 Berkeley CA',
 'Email_Addr': nan,
 'Phone': nan,
 'Subclass': {'Employee': {'Employee_ID': 9,
   'SSN': 123423,
   'Store': 3,
   'Union': 1,
   'Working_Days': ['M', 'F'],
   'Appraiser': 7,
   'Store_staff': 7},
  'Consignor': {},
  'Customer': {'Customer_ID': 45, 'Membership': 5456323},
  'Designer': {'Designer_ID': 47}}}

##### Query 4: find all employees that are also customers

In [32]:
q4 = {
  'Subclass.Employee': {
      '$gt': {}
  },
  'Subclass.Customer': {
      '$gt': {}
  },
}

In [33]:
for q_result in mdb_coll_pers.find(q4):
    display(q_result)

{'_id': ObjectId('64506188a071c96ae9396d23'),
 'PID': 1,
 'FName': 'John',
 'MI': 'R',
 'LName': 'Graft',
 'Address': 'Milvia 234 Berkeley CA',
 'Email_Addr': 'jrg@me.com',
 'Phone': nan,
 'Subclass': {'Employee': {'Employee_ID': '1',
   'SSN': '123',
   'Store': '1',
   'Union': '1',
   'Working_Days': ['M', 'W'],
   'Appraiser': '1',
   'Store_staff': '1'},
  'Consignor': {},
  'Customer': {'Customer_ID': '1', 'Membership': '543'},
  'Designer': {}}}

{'_id': ObjectId('64506188a071c96ae9396d25'),
 'PID': 3,
 'FName': 'Don',
 'MI': 'D',
 'LName': 'Perez',
 'Address': 'Shattuck 434 Berkeley CA',
 'Email_Addr': 'don@yahoo.com',
 'Phone': nan,
 'Subclass': {'Employee': {'Employee_ID': 3,
   'SSN': 1245,
   'Store': 1,
   'Union': 1,
   'Working_Days': ['F', 'T'],
   'Appraiser': 3,
   'Store_staff': 3},
  'Consignor': {},
  'Customer': {'Customer_ID': 2, 'Membership': 998},
  'Designer': {}}}

{'_id': ObjectId('64506188a071c96ae9396d28'),
 'PID': 6,
 'FName': 'John',
 'MI': 'Z',
 'LName': 'Holz',
 'Address': 'Milvia 452 Berkeley CA',
 'Email_Addr': nan,
 'Phone': nan,
 'Subclass': {'Employee': {'Employee_ID': 9,
   'SSN': 123423,
   'Store': 3,
   'Union': 1,
   'Working_Days': ['M', 'F'],
   'Appraiser': 7,
   'Store_staff': 7},
  'Consignor': {},
  'Customer': {'Customer_ID': 45, 'Membership': 5456323},
  'Designer': {'Designer_ID': 47}}}

### 

# Database cleaning

##### Dropping collections

In [9]:
mdb.list_collection_names()

['person', 'online_purchase_data', 'product', 'inventory']

In [10]:
mdb_coll_pers.drop()

In [11]:
mdb.list_collection_names()

['online_purchase_data', 'product', 'inventory']

##### Cleaning database 

In [ ]:
print(mdb_client.list_database_names())

In [ ]:
mdb_client.drop_database('beans_boutique_db')

In [ ]:
print(mdb_client.list_database_names())

### 

# Header

### 

# *Notes*

---

---